In [12]:
##########################################################################################
############################### 1) With Embedding ########################################

import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed

import numpy as np
embed_dim = 128
data_dim = 1
timesteps = 25
vocab_size = 8100

x_old = np.load('idx_q.npy')
y_old = np.load('idx_a.npy')

keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

embedding = Sequential()
embedding.add(Embedding(vocab_size,embed_dim,input_length=timesteps))
embedding.compile('rmsprop', 'mse')

x = embedding.predict(x_old)
y = embedding.predict(y_old)

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
#model.add(Embedding(8100,embed_dim,input_length=timesteps))
model.add(LSTM(embed_dim,input_shape=(timesteps,embed_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(embed_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(embed_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
#model.add(TimeDistributed(Dense(1,activation='relu')))


model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy']
              )

# generate dummy training data


x_train = x[1:140001].reshape(140000,timesteps,embed_dim)
y_train = y[1:140001].reshape(140000,timesteps,embed_dim)
# generate dummy validation data
x_val = x[140001:150000].reshape(9999,timesteps,embed_dim)
y_val = y[140001:150000].reshape(9999,timesteps,embed_dim)
model.fit(x_train, y_train,
          batch_size=128, nb_epoch=20,
          validation_data=(x_val, y_val))
model.save('my_model.h5')
x_test = x[150001:160000].reshape(9999,timesteps,embed_dim)
y_test = y[150001:160000].reshape(9999,timesteps,embed_dim)
score = model.evaluate(x_test, y_test, batch_size=64)
print("\nModel Accuracy: %.2f%%" % (score[1]*100))

AttributeError: 'Sequential' object has no attribute 'load'

In [5]:
print(score)

[480337.9356212425, 0.012264528604810367]


In [2]:
c=model.predict(x[40001:40500].reshape(499,25))

In [3]:
print(c[1])

[ 0.09884203  0.08344298  0.09658517  0.09359372  0.08724034  0.07723846
  0.0680349   0.06117992  0.05216976  0.04656862  0.03937593  0.03422173
  0.03021572  0.02661941  0.02194737  0.01883822  0.01598314  0.0128208
  0.01082024  0.00868252  0.00629053  0.00474371  0.00292437  0.00145011
  0.00017013]


In [ ]:
##########################################################################################
############################ 2) Normal LSTM No Embedding #################################

import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed

import numpy as np
embed_dim = 128
data_dim = 1
timesteps = 25
vocab_size = 8100

x = np.load('idx_q.npy')
y = np.load('idx_a.npy')

keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

#embedding = Sequential()
#embedding.add(Embedding(vocab_size,embed_dim,input_length=timesteps))
#embedding.compile('rmsprop', 'mse')

#x = embedding.predict(x_old)
#y = embedding.predict(y_old)
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(data_dim,input_shape=(timesteps,data_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(data_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(data_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
#model.add(TimeDistributed(Dense(embed_dim,activation='relu')))
#model.add(TimeDistributed(Dense(data_dim,activation='relu')))
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy']
              )

# generate dummy training data


x_train = x[1:140001].reshape(140000,timesteps,data_dim)
y_train = y[1:140001].reshape(140000,timesteps,data_dim)
# generate dummy validation data
x_val = x[140001:150000].reshape(9999,timesteps,data_dim)
y_val = y[140001:150000].reshape(9999,timesteps,data_dim)
model.fit(x_train, y_train,
          batch_size=128, nb_epoch=20,
          validation_data=(x_val, y_val))
model.save('my_model.h5')
x_test = x[150001:160000].reshape(9999,timesteps,data_dim)
y_test = y[150001:160000].reshape(9999,timesteps,data_dim)
score = model.evaluate(x_test, y_test, batch_size=64)
print("\nModel Accuracy: %.2f%%" % (score[1]*100))

Using TensorFlow backend.


Train on 140000 samples, validate on 9999 samples
Epoch 1/20
140000/140000 [==============================] - 186s - loss: 402546.0464 - acc: 0.1487 - val_loss: 432623.3359 - val_acc: 0.0098
Epoch 2/20
140000/140000 [==============================] - 177s - loss: 402465.1430 - acc: 0.0107 - val_loss: 432610.5873 - val_acc: 0.0110
Epoch 3/20
 14208/140000 [==>...........................] - ETA: 149s - loss: 399385.0842 - acc: 0.0108

In [ ]:
##########################################################################################
################################# 3) LSTM One hot ########################################

import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed

import numpy as np
embed_dim = 128
data_dim = 1
timesteps = 25


x = np.load('idx_q.npy')
y_old = np.load('idx_a.npy')
vocab_size = np.amax(y_old)+1
y=(np.arange(y_old.max()) == y_old[:,:,None]-1).astype(int)
keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

#embedding = Sequential()
#embedding.add(Embedding(vocab_size,embed_dim,input_length=timesteps))
#embedding.compile('rmsprop', 'mse')

#x = embedding.predict(x_old)
#y = embedding.predict(y_old)
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(data_dim,input_shape=(timesteps,data_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(data_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(data_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(TimeDistributed(Dense(vocab_size,activation='softmax')))
#model.add(TimeDistributed(Dense(data_dim,activation='relu')))
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy']
              )

# generate dummy training data


x_train = x[1:140001].reshape(140000,timesteps,data_dim)
y_train = y[1:140001].reshape(140000,timesteps,vocab_size)
# generate dummy validation data
x_val = x[140001:150000].reshape(9999,timesteps,data_dim)
y_val = y[140001:150000].reshape(9999,timesteps,vocab_size)
model.fit(x_train, y_train,
          batch_size=128, nb_epoch=20,
          validation_data=(x_val, y_val))
model.save('my_model.h5')
x_test = x[150001:160000].reshape(9999,timesteps,data_dim)
y_test = y[150001:160000].reshape(9999,timesteps,vocab_size)
score = model.evaluate(x_test, y_test, batch_size=64)
print("\nModel Accuracy: %.2f%%" % (score[1]*100))

Using TensorFlow backend.


Train on 140000 samples, validate on 9999 samples
Epoch 1/20
 70272/140000 [==============>...............] - ETA: 141s - loss: 389574.3568 - acc: 0.0036

In [ ]:
##########################################################################################
################################# 4) LSTM Time distributed dense #########################
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed

import numpy as np
embed_dim = 128
data_dim = 1
timesteps = 25

keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0)
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
#model.add(Embedding(8100,embed_dim,input_length=timesteps))
model.add(LSTM(data_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(data_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
#model.add(LSTM(data_dim, dropout_W=0.2, dropout_U=0.2))
model.add(TimeDistributed(Dense(data_dim,activation='relu')))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy']
              )

# generate dummy training data
x = np.load('idx_q.npy')
y = np.load('idx_a.npy')

x_train = x[1:140001].reshape(140000,25)
y_train = y[1:140001].reshape(140000,25,1)
# generate dummy validation data
x_val = x[140001:150000].reshape(9999,25)
y_val = y[140001:150000].reshape(9999,25,1)
model.fit(x_train, y_train,
          batch_size=128, nb_epoch=20,
          validation_data=(x_val, y_val))
model.save('my_model.h5')
x_test = x[150001:160000].reshape(9999,25)
y_test = y[150001:160000].reshape(9999,25,1)
score = model.evaluate(x_test, y_test, batch_size=64)
print("\nModel Accuracy: %.2f%%" % (score[1]*100))

In [10]:
print(y_train.shape)

(25000,)


In [8]:
c = model.predict(X_test,batch_size=batch_size)

In [9]:
print(c[1])

[ 0.29208347]


In [14]:
import numpy as np
import tensorflow as tf
x = np.load('idx_q.npy')
y = np.load('idx_a.npy')
print(x.shape)
a=np.array([[1, 7, 5, 3], # Modified the sample listed in question for variety
       [2, 4, 1, 4]])

#b=(np.arange(y.max()) == y[:,:,None]-1).astype(int)
#b=np.eye(8100)[y]
print(np.amax(y))

(166559, 25)
8001
